## A demo using Hydrogen Hamiltonian with GPT-QE.

In [1]:
import torch
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader
from gqe.mingpt.utils import set_seed

set_seed(31)

In [2]:
from qwrapper.operator import PauliObservable
from gqe.mingpt.cost import EnergyCost
from qswift.compiler import DefaultOperatorPool
from benchmark.molecule import DiatomicMolecularHamiltonian
from gqe.operator_pool.uccsd import UCCSD, generate_molecule
from gqe.common.initializer import HFStateInitializer
from gqe.mingpt.callback import DefaultCallback, PrintMonitor, FileMonitor
from gqe.util import get_device

#molecule = generate_molecule("H", "H", 0.7414, "sto-3g")
molecule = generate_molecule("H", "H", 1.5, "sto-3g", bravyi_kitaev=False)
nqubit = 4

# prepare Hamiltonian
hamiltonian = DiatomicMolecularHamiltonian(nqubit, molecule, bravyi_kitaev=False)

# prepare operator_pool
uccsd = UCCSD(nqubit, molecule)
paulis = uccsd.paulis
paulis.append(PauliObservable("IIII"))
print('paulis', paulis)
num_operators = len(paulis)
initializer = HFStateInitializer(n_electrons=2)
pool = DefaultOperatorPool(paulis)
cost = EnergyCost(hamiltonian, initializer, pool,
                  [0.00625, -0.00625, 0.0125, -0.0125, 0.025, -0.025, 0.05,
                   -0.05, 0.1, -0.1], device=get_device())
print(hamiltonian.exact_value(initializer.init_circuit(4, [], "qulacs")))

converged SCF energy = -0.910873554594387
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
paulis [+IYZX, +XXYX, +XYYY, +YXYY, +YYYX, +YZXI, +IIII]
-0.9108735545943867


## FCI energy by diagonalization

In [3]:
from qwrapper.hamiltonian import compute_ground_state

print(compute_ground_state(hamiltonian))

-0.9981493534714105


## Setup for GPT

In [4]:
# create a GPT instance
from gqe.mingpt.model import GPT

model_config = GPT.get_default_config()
model_config.model_type = 'gpt2'
model_config.vocab_size = cost.vocab_size()
model_config.block_size = cost.vocab_size()
model_config.n_gates = 10  # The number of gates for each circuit
model_config.temperature = 5  # Each gate is generated with probability exp(-temperature * logit)
model_config.embd_pdrop = 0
model_config.resid_pdrop = 0
model_config.attn_pdrop = 0
model = GPT(model_config, cost)

number of parameters: 85.16M


In [5]:
# create a Trainer object
from gqe.mingpt.trainer import Trainer

train_config = Trainer.get_default_config()
train_config.learning_rate = 5e-7  # the model we're using is so small that we can go a bit faster
train_config.max_iters = 100
train_config.num_workers = 0
train_config.n_samples = 10
trainer = Trainer(train_config, model)

running on device mps


In [6]:
file_monitor = FileMonitor()
callback_generator = DefaultCallback(model, monitors=[PrintMonitor(), file_monitor])

trainer.set_callback('on_batch_end', callback_generator.generate())
trainer.run()
torch.save(model.state_dict(), '../saved_models/gptqe_test_2')
file_monitor.save('../output/test_batch.json')

iter_dt 0.00s; iter 0: train loss 0.24785 temperature: 5
mean_logits tensor([-0.9802, -0.9943, -1.3139, -1.0694, -1.0196, -1.1741, -1.0406, -1.1028,
        -1.1996, -1.0560], device='mps:0', grad_fn=<SubBackward0>)
energies: tensor([-0.9198, -0.8957, -0.9768, -0.9838, -0.9489, -0.9911, -0.9446, -0.9254,
        -0.9681, -0.9436], device='mps:0')
[43, 0, 43, 27, 43, 65, 61, 42, 47, 3]
[43, 33, 43, 27, 65, 61, 45, 61, 65, 43]
[43, 65, 43, 4, 43, 43, 4, 43, 43, 33]
[43, 0, 43, 45, 59, 43, 4, 61, 33, 33]
[0, 43, 15, 43, 65, 61, 43, 33, 43, 33]
[43, 43, 0, 43, 65, 43, 43, 17, 43, 43]
[43, 43, 4, 65, 43, 4, 4, 3, 43, 4]
[43, 43, 27, 53, 65, 65, 61, 24, 43, 43]
[43, 43, 4, 33, 43, 65, 43, 33, 17, 43]
[43, 43, 27, 43, 61, 33, 12, 43, 65, 33]
mean: tensor(-0.9498, device='mps:0')
current min: -0.9911292
iter_dt 1696206398.39s; iter 1: train loss 0.14540 temperature: 5.05
mean_logits tensor([-0.9693, -0.7377, -0.6616, -0.9145, -0.9531, -0.8059, -0.6750, -0.9566,
        -0.8041, -0.8950], devic

KeyboardInterrupt: 

In [ ]:
indices, logits = model.generate(torch.tensor([[0]]).to(get_device()), model_config.n_gates)
print(cost.energy(indices))

In [ ]:
model.temperature = 20
model.load_state_dict(torch.load('../saved_models/gptqe_test_2'))
indices, logits = model.generate(torch.zeros(10, 1, dtype=torch.int).to(get_device()), model_config.n_gates)
cost.sequence.tool = "qiskit"
index = torch.argmin(cost.energy(indices)).item()

In [ ]:
target = indices.cpu().numpy()[index]
print(cost.energy(torch.tensor([target]).to(get_device())))
print(target)
cost.sequence.get_circuit(target).qc.draw(output="mpl", plot_barriers=True)